In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense, Conv2D , Conv1D, MaxPool1D, MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
import joblib
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import re
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Dropout, Dense, Flatten, Input, Lambda
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau , EarlyStopping
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras import regularizers
import tensorflow as tf
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import albumentations as A

transform = A.Compose([
    A.RandomBrightnessContrast(),
    A.Affine(scale=[0.8,1.2],translate_percent=0.05, shear=0.2, keep_ratio=True, p=0.5),
    A.Rotate(limit=10)
])

## Without mediapipe

#### Read image and convert it to 28x28 matrix:

In [ ]:
folder_name = '/content/drive/MyDrive/IS4242/data' #might be different for different people
path = folder_name
img_list = []
label_list = []
txt = 'abcdefghiklmnopqrstuvwxy'

for i in tqdm(range(len(os.listdir(path)))):
  filename = os.listdir(path)[i]
  if filename.endswith(('.jpg','.png')):
      label_name = re.split(r'[-_]', filename)[0].lower()
      if label_name not in txt:
        continue
      label_list.append(label_name)
      img = Image.open(os.path.join(path, filename))
      img = img.resize((28, 28), Image.ANTIALIAS)
      img = img.convert('L')
      img_array = np.array(img)
      
      img_list.append(img_array)


In [ ]:
X = np.array(img_list)

y = np.array(label_list)
y = np.array(list(map(lambda x:txt.find(x), y)))
y = to_categorical(y)

#### Train test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

output_dir = '/content/drive/MyDrive/IS4242/dump/'
joblib.dump(X, output_dir+'X.pkl')
joblib.dump(y, output_dir+'y.pkl')

(634, 28, 28)
(159, 28, 28)
(634, 24)
(159, 24)


['/content/drive/MyDrive/IS4242/dump/y.pkl']

#### Augmentation only on train data

In [ ]:
augmented_X = []
augmented_y = []
for i in range(len(X_train)):
  for j in (range(10)):
      transformed = transform(image=X_train[i])
      augmented_X.append(transformed['image'])
      augmented_y.append(y_train[i])

augmented_X = np.array(augmented_X)
augmented_y = np.array(augmented_y)
X_train = np.concatenate([X_train,augmented_X])
y_train = np.concatenate([y_train,augmented_y])

output_dir = '/content/drive/MyDrive/IS4242/dump/'
joblib.dump(augmented_X, output_dir+'augmented_X.pkl')
joblib.dump(augmented_y, output_dir+'augmented_y.pkl')

['/content/drive/MyDrive/IS4242/dump/augmented_y.pkl']

In [ ]:
print(X_train.shape)
print(y_train.shape)

(6974, 28, 28)
(6974, 24)


#### Reshape and divide the pixels by 255

In [ ]:
X_train = X_train/255.0
X_test = X_test/255.0
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(6974, 28, 28, 1)
(159, 28, 28, 1)
(6974, 24)
(159, 24)


#### CNN with augmentation

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,
        brightness_range=[0.8,1.2],
        vertical_flip=False)


datagen.fit(X_train)

In [ ]:
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
# architecture
# Set the CNN model 

model = Sequential()
model.add(Conv2D(filters = 128, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 128, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation = "relu"))
model.add(Dense(256, activation = "relu"))
model.add(Dense(24, activation = "softmax"))

model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_42 (Conv2D)          (None, 28, 28, 32)        832       
                                                                 
 conv2d_43 (Conv2D)          (None, 28, 28, 32)        25632     
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 14, 14, 32)       0         
 g2D)                                                            
                                                                 
 batch_normalization_21 (Bat  (None, 14, 14, 32)       128       
 chNormalization)                                                
                                                                 
 dropout_28 (Dropout)        (None, 14, 14, 32)        0         
                                                                 
 conv2d_44 (Conv2D)          (None, 14, 14, 64)       

In [ ]:
epochs = 30
batch_size = 32


model_checkpoint_callback = ModelCheckpoint(
    filepath="best.h5",
    monitor='val_accuracy', 
    verbose=1, 
    save_best_only=True, 
    mode='max')

es = EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=10)

# Fit the model
history = model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_test, y_test),steps_per_epoch=X_train.shape[0] // batch_size,
                              verbose = 1,   callbacks=[model_checkpoint_callback,learning_rate_reduction,es])

Epoch 1/30


<ipython-input-169-021abc8791b3>:15: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size),


54/54 [==============================] - ETA: 0s - loss: 3.4244 - accuracy: 0.0540
Epoch 1: val_accuracy improved from -inf to 0.01887, saving model to best.h5
54/54 [==============================] - 81s 1s/step - loss: 3.4244 - accuracy: 0.0540 - val_loss: 3.2231 - val_accuracy: 0.0189 - lr: 0.0010
Epoch 2/30
54/54 [==============================] - ETA: 0s - loss: 3.1336 - accuracy: 0.0711
Epoch 2: val_accuracy improved from 0.01887 to 0.06289, saving model to best.h5
54/54 [==============================] - 82s 2s/step - loss: 3.1336 - accuracy: 0.0711 - val_loss: 3.2113 - val_accuracy: 0.0629 - lr: 0.0010
Epoch 3/30
54/54 [==============================] - ETA: 0s - loss: 3.0425 - accuracy: 0.0963
Epoch 3: val_accuracy did not improve from 0.06289
54/54 [==============================] - 77s 1s/step - loss: 3.0425 - accuracy: 0.0963 - val_loss: 3.2515 - val_accuracy: 0.0440 - lr: 0.0010
Epoch 4/30
54/54 [==============================] - ETA: 0s - loss: 2.8801 - accuracy: 0.1382
E

In [ ]:
results = model.predict(X_test)
y_pred = np.argmax(results,axis = 1) 
y_true = np.argmax(y_test,axis = 1) 


5/5 [==============================] - 0s 71ms/step


#### Results

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.50      0.40      0.44         5
           1       0.25      0.40      0.31         5
           2       0.83      0.42      0.56        12
           3       0.50      0.40      0.44         5
           4       0.38      0.43      0.40         7
           5       0.57      0.67      0.62         6
           6       0.00      0.00      0.00         5
           7       0.25      0.09      0.13        11
           8       0.75      0.60      0.67         5
           9       0.75      0.50      0.60         6
          10       0.67      0.67      0.67         3
          11       0.00      0.00      0.00         3
          12       0.11      0.20      0.14         5
          13       0.57      0.50      0.53         8
          14       0.67      0.57      0.62         7
          15       0.33      0.17      0.22         6
          16       0.29      0.25      0.27         8
          17       0.50    

#### Prediction and evaluation

In [ ]:
img = Image.open('m.png')
img = img.resize((28, 28), Image.ANTIALIAS)
img = img.convert('L')
img_array = np.array(img)
img_array = img_array/255.0
img_array = img_array.reshape(-1, 28, 28, 1)

In [ ]:
results = model.predict(img_array)
y_pred = np.argmax(results,axis = 1)
txt[y_pred[0]]

1/1 [==============================] - 0s 46ms/step


'm'

#### After hyperparameter tuning, train it with the whole data

In [ ]:
output_dir = '/content/drive/MyDrive/IS4242/dump/'
X = joblib.load(output_dir+'X.pkl')
y = joblib.load(output_dir+'y.pkl')

In [ ]:
augmentX = []
augmenty = []
for i in range(len(X)):
  for j in (range(10)):
      transformed = transform(image=X[i])
      augmentX.append(transformed['image'])
      augmenty.append(y[i])

augmentX = np.array(augmentX)
augmenty = np.array(augmenty)
X = np.concatenate([X,augmentX])
y = np.concatenate([y,augmenty])

In [ ]:
X = X/255.0
X = X.reshape(-1, 28, 28, 1)

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,
        brightness_range=[0.8,1.2],
        vertical_flip=False)


datagen.fit(X)

In [ ]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2) , padding = 'same'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2) , padding = 'same'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , padding = 'same'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dense(512, activation = "relu"))
model.add(Dense(24, activation = "softmax"))

model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
epochs = 30
batch_size = 64

# Fit the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit_generator(datagen.flow(X, y, batch_size=batch_size),epochs = epochs)

In [ ]:
img = Image.open('m.png')
img = img.resize((28, 28), Image.ANTIALIAS)
img = img.convert('L')
img_array = np.array(img)
img_array = img_array/255.0
img_array = img_array.reshape(-1, 28, 28, 1)

In [ ]:
results = model.predict(img_array)
y_pred = np.argmax(results,axis = 1)
txt[y_pred[0]]

1/1 [==============================] - 0s 46ms/step


'm'

## Using mediapipe

In [ ]:
!pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.6/33.6 MB 42.0 MB/s eta 0:00:00


In [ ]:
# For capturing hand coordinates
import cv2
import tensorflow as tf
import mediapipe as mp

In [ ]:
# For processing data
import csv
import os
import numpy as np
import pandas as pd
from tqdm import tqdm 
import re

#### Convert image data to matrix

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
import albumentations as A

transform = A.Compose([
    A.RandomBrightnessContrast(),
    A.Affine(scale=[0.8,1.2],translate_percent=0.05, shear=0.2, keep_ratio=True, p=0.5),
    A.Rotate(limit=10)
])

In [ ]:
folder_name = '/content/drive/MyDrive/IS4242/data' #might be different for different people
path = folder_name
img_list = []
label_list = []
txt = 'abcdefghiklmnopqrstuvwxy'

for i in tqdm(range(len(os.listdir(path)))):
  filename = os.listdir(path)[i]
  if filename.endswith(('.jpg','.png')):
      label_name = re.split(r'[-_]', filename)[0].lower()
      if label_name not in txt:
        continue
      label_list.append(label_name)
      img = Image.open(os.path.join(path, filename))
      img = img.resize((224, 224), Image.ANTIALIAS) # bigger size
      img = img.convert('L')
      img_array = np.array(img)
      
      img_list.append(img_array)

X = np.array(img_list)

y = np.array(label_list)
y = np.array(list(map(lambda x:txt.find(x), y)))
y = to_categorical(y)

output_dir = '/content/drive/MyDrive/IS4242/dump/'
joblib.dump(X,output_dir+'X_224.pkl')
joblib.dump(y,output_dir+'y_224.pkl')


 16%|█▌        | 196/1221 [00:26<03:02,  5.63it/s]

In [ ]:
output_dir = '/content/drive/MyDrive/IS4242/dump/'
X = joblib.load(output_dir+'X_224.pkl')
y = joblib.load(output_dir+'y_224.pkl')

In [ ]:
len(X)

817

#### Train test split

In [ ]:
from sklearn.model_selection import train_test_split

# We will take 33% from 1000 for our test data.
# Recommended value 80:20, 67:33, 50:50
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Augmentation only on train data

In [ ]:
X_train

array([[[212, 215, 214, ..., 220, 217, 209],
        [227, 218, 213, ..., 212, 208, 217],
        [185, 174, 181, ..., 164, 197, 205],
        ...,
        [232, 232, 232, ..., 145, 129, 148],
        [232, 232, 232, ..., 150, 138, 138],
        [232, 232, 232, ..., 159, 129, 124]],

       [[147, 148, 149, ..., 142, 142, 143],
        [147, 148, 149, ..., 141, 141, 142],
        [147, 148, 149, ..., 144, 144, 145],
        ...,
        [140, 139, 139, ..., 133, 134, 133],
        [140, 139, 139, ..., 133, 134, 133],
        [140, 139, 139, ..., 131, 133, 133]],

       [[183, 182, 181, ..., 175, 172, 175],
        [183, 180, 180, ..., 174, 174, 175],
        [181, 181, 181, ..., 174, 174, 174],
        ...,
        [171, 171, 171, ...,  32,  33,  33],
        [172, 171, 171, ...,  29,  28,  28],
        [171, 173, 173, ...,  28,  28,  29]],

       ...,

       [[  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0

In [ ]:
augmented_X = []
augmented_y = []
for i in range(len(X_train)):
  for j in (range(10)):
      transformed = transform(image=X_train[i])
      augmented_X.append(transformed['image'])
      augmented_y.append(y_train[i])

augmented_X = np.array(augmented_X)
augmented_y = np.array(augmented_y)
X_train = np.concatenate([X_train,augmented_X])
y_train = np.concatenate([y_train,augmented_y])

#### Generate dataset of hand points

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [ ]:
data = []
labels = []

In [ ]:
count = 0
with mp_hands.Hands(static_image_mode =True, max_num_hands = 2, min_detection_confidence=0.5) as hands:
    for i in tqdm(range(len(X_train))):
        # Load the image and extract its features
            results = hands.process(cv2.cvtColor(cv2.flip(X_train[i],1), cv2.COLOR_BGR2RGB))
            try:
                # Extract Hand landmarks
                for hand_landmark in results.multi_hand_landmarks:
                    right_hand = hand_landmark.landmark
                right_hand_row = list(np.array([[landmark.x, landmark.y] for landmark in right_hand]).flatten())
                # Concate rows
                row = right_hand_row

                # Extract the label from the image filename (e.g. "A.jpg")
                label = y_train[i]

                data.append(row)
                labels.append(label)
                print(row, label)
            except Exception as e:
                count += 1
                continue


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
count

4730

In [ ]:
pd.DataFrame(data)

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0.570230,0.700385,0.460918,0.601834,0.385230,0.526549,0.322623,0.470859,0.301588,0.408549,...,0.434779,0.504095,0.549578,0.469310,0.416643,0.467967,0.423916,0.518278,0.466331,0.533848
1,0.446183,0.796893,0.341452,0.681229,0.304357,0.500339,0.314089,0.366598,0.315794,0.253817,...,0.488412,0.601652,0.734732,0.564161,0.699286,0.469736,0.620621,0.544259,0.574593,0.615079
2,0.554823,0.467337,0.477951,0.500206,0.370337,0.497704,0.303738,0.458727,0.319072,0.396847,...,0.466813,0.410920,0.478718,0.277710,0.441688,0.218711,0.420079,0.179862,0.393535,0.143242
3,0.421717,0.593616,0.332333,0.543942,0.278878,0.457589,0.282298,0.384428,0.314716,0.334537,...,0.391403,0.521059,0.528068,0.395230,0.475645,0.423229,0.459146,0.468935,0.459199,0.488524
4,0.651359,0.619728,0.656911,0.514619,0.602316,0.457691,0.539696,0.429281,0.491382,0.435781,...,0.391995,0.521948,0.437633,0.637292,0.367256,0.589515,0.384333,0.575371,0.412115,0.575475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2448,0.583231,0.452619,0.539714,0.455225,0.495395,0.492904,0.479851,0.551855,0.483757,0.602373,...,0.545168,0.576425,0.487555,0.524171,0.515908,0.578480,0.545968,0.580548,0.566784,0.575146
2449,0.556153,0.509525,0.514526,0.529332,0.459658,0.563573,0.428268,0.608088,0.417417,0.649978,...,0.471311,0.668249,0.432259,0.565127,0.447259,0.629455,0.473636,0.645954,0.497372,0.653503
2450,0.583231,0.452619,0.539714,0.455225,0.495395,0.492904,0.479851,0.551855,0.483757,0.602373,...,0.545168,0.576425,0.487555,0.524171,0.515908,0.578480,0.545968,0.580548,0.566784,0.575146
2451,0.583231,0.452619,0.539714,0.455225,0.495395,0.492904,0.479851,0.551855,0.483757,0.602373,...,0.545168,0.576425,0.487555,0.524171,0.515908,0.578480,0.545968,0.580548,0.566784,0.575146


In [ ]:
import joblib

output_dir = f"/content/drive/MyDrive/IS4242/dump/"
joblib.dump(data, output_dir+'data_svm_train.pkl')
joblib.dump(labels, output_dir+'labels_svm_train.pkl')

['/content/drive/MyDrive/IS4242/dump/labels_svm_train.pkl']

In [ ]:
from sklearn.preprocessing import StandardScaler
import joblib

scaler = StandardScaler().fit(data)

output_dir = f"/content/drive/MyDrive/IS4242/dump/"
joblib.dump(scaler, output_dir+'scaler_svm.pkl')



['/content/drive/MyDrive/IS4242/dump/scaler_svm.pkl']

In [ ]:
import joblib
output_dir = f"/content/drive/MyDrive/IS4242/dump/"
data = joblib.load(output_dir+'data_knn_2.pkl')
labels = joblib.load(output_dir+'labels_knn_2.pkl')

In [ ]:
test_data = []
test_label = []
count = 0
with mp_hands.Hands(static_image_mode =True, max_num_hands = 2, min_detection_confidence=0.5) as hands:
    for i in tqdm(range(len(X_test))):
        # Load the image and extract its features
            results = hands.process(cv2.cvtColor(cv2.flip(X_test[i],1), cv2.COLOR_BGR2RGB))
            try:
                # Extract Hand landmarks
                for hand_landmark in results.multi_hand_landmarks:
                    right_hand = hand_landmark.landmark
                right_hand_row = list(np.array([[landmark.x, landmark.y] for landmark in right_hand]).flatten())
                # Concate rows
                row = right_hand_row

                # Extract the label from the image filename (e.g. "A.jpg")
                label = y_test[i]

                test_data.append(row)
                test_label.append(label)
                print(row, label)
            except Exception as e:
                count += 1
                continue

  3%|▎         | 5/164 [00:00<00:07, 22.52it/s]

[0.5210281014442444, 0.641315221786499, 0.39546293020248413, 0.5578718781471252, 0.3307020962238312, 0.3800443112850189, 0.3052407503128052, 0.24902814626693726, 0.2702046036720276, 0.1634766012430191, 0.41695016622543335, 0.2820096015930176, 0.4198142886161804, 0.16589711606502533, 0.42496901750564575, 0.27493152022361755, 0.43036800622940063, 0.339459091424942, 0.5108478665351868, 0.2845029830932617, 0.5148842334747314, 0.17844367027282715, 0.4996938705444336, 0.3112812340259552, 0.49929216504096985, 0.34388941526412964, 0.5973529815673828, 0.31039708852767944, 0.6130174398422241, 0.20091238617897034, 0.5859422087669373, 0.32097747921943665, 0.5842393040657043, 0.3630259037017822, 0.6818413734436035, 0.3520607650279999, 0.6982240676879883, 0.2732140123844147, 0.6630502939224243, 0.35034728050231934, 0.6547291278839111, 0.3847343325614929] [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.5496582388877869, 0.6345279216766357, 0.47247281670570374, 0.625109791

  7%|▋         | 11/164 [00:00<00:06, 23.93it/s]

[0.5303622484207153, 0.7261391878128052, 0.3620838522911072, 0.6153806447982788, 0.2671659290790558, 0.40738582611083984, 0.22772908210754395, 0.24531611800193787, 0.17089855670928955, 0.12447883188724518, 0.40584099292755127, 0.294745534658432, 0.34948059916496277, 0.20392224192619324, 0.34238922595977783, 0.3307180404663086, 0.36016517877578735, 0.4387580454349518, 0.5203278064727783, 0.30517739057540894, 0.4636331796646118, 0.20133087038993835, 0.451718270778656, 0.3622964024543762, 0.46842825412750244, 0.47976988554000854, 0.6406235694885254, 0.32791975140571594, 0.569840133190155, 0.22479650378227234, 0.5453921556472778, 0.3856639564037323, 0.5597202777862549, 0.5043128728866577, 0.764388918876648, 0.36453112959861755, 0.6879178285598755, 0.2666337490081787, 0.6525718569755554, 0.3518322706222534, 0.6621406674385071, 0.42191678285598755] [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.5895628929138184, 0.7734596133232117, 0.6003317832946777, 0.66376382

 13%|█▎        | 22/164 [00:00<00:04, 29.03it/s]

...

 90%|████████▉ | 147/164 [00:06<00:00, 18.35it/s]

[0.4025712013244629, 0.5869166851043701, 0.4375220537185669, 0.522254228591919, 0.5011380314826965, 0.4978063702583313, 0.5560323596000671, 0.5066971778869629, 0.5907210111618042, 0.523162841796875, 0.5285419225692749, 0.5317249298095703, 0.594545841217041, 0.5259853601455688, 0.6381144523620605, 0.5236724019050598, 0.6700278520584106, 0.5221999287605286, 0.5290641784667969, 0.5704972743988037, 0.5946837663650513, 0.5639264583587646, 0.6356530785560608, 0.556513249874115, 0.6702843308448792, 0.5516738891601562, 0.5267694592475891, 0.6048494577407837, 0.5903812050819397, 0.596366822719574, 0.632245659828186, 0.5849146842956543, 0.6661837100982666, 0.5781964659690857, 0.522779643535614, 0.6322373151779175, 0.5783458948135376, 0.6181328296661377, 0.6143463253974915, 0.6030421853065491, 0.643071711063385, 0.5925341248512268] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[0.659817636013031, 0.7055771350860596, 0.7152523994445801, 0.5528687238693237, 0.67378699779

 94%|█████████▍| 154/164 [00:07<00:00, 18.76it/s]

[0.47691941261291504, 0.7220901846885681, 0.4109916090965271, 0.6569188833236694, 0.4090764820575714, 0.5418635010719299, 0.47629427909851074, 0.4535200595855713, 0.537406325340271, 0.3900882601737976, 0.37297624349594116, 0.46395376324653625, 0.38338690996170044, 0.41171514987945557, 0.375829815864563, 0.48565569519996643, 0.363042950630188, 0.5369953513145447, 0.4320586621761322, 0.46602949500083923, 0.44608330726623535, 0.4027238190174103, 0.4247105121612549, 0.49308809638023376, 0.3995841145515442, 0.5473983287811279, 0.4899352788925171, 0.4852662682533264, 0.506833553314209, 0.4309278130531311, 0.47783714532852173, 0.5107895135879517, 0.4487495720386505, 0.5549561381340027, 0.542538046836853, 0.515808641910553, 0.5471177697181702, 0.48352673649787903, 0.5181245803833008, 0.5462149977684021, 0.4944431781768799, 0.5775335431098938] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


100%|██████████| 164/164 [00:07<00:00, 20.96it/s]


In [ ]:
import joblib

output_dir = f"/content/drive/MyDrive/IS4242/dump/"
joblib.dump(test_data, output_dir+'data_svm_test.pkl')
joblib.dump(test_label, output_dir+'labels_svm_test.pkl')

['/content/drive/MyDrive/IS4242/dump/labels_svm_test.pkl']

In [ ]:
labels = np.array(labels)
test_label = np.array(test_label)

In [ ]:
x_train = np.array(data)
x_test = np.array(test_data)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
print(x_train.shape)
print(x_test.shape)

(2453, 42, 1)
(44, 42, 1)


#### SVM - linear


In [ ]:
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import confusion_matrix

In [ ]:
# linear model

model_linear = SVC(kernel='linear')
model_linear.fit(x_train.reshape((x_train.shape[0],42)), np.argmax(labels, axis=1))

# predict
y_pred = model_linear.predict(x_test.reshape((x_test.shape[0],42)))

##### Evaluation

In [ ]:
# confusion matrix and accuracy

# accuracy
print("accuracy:", metrics.accuracy_score(y_true=np.argmax(test_label, axis=1), y_pred=y_pred), "\n")

# cm
print(metrics.confusion_matrix(y_true=np.argmax(test_label, axis=1), y_pred=y_pred))

accuracy: 0.8409090909090909 

[[3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(np.argmax(test_label, axis=1), y_pred))

              precision    recall  f1-score   support

           0       0.75      1.00      0.86         3
           1       1.00      1.00      1.00         1
           2       1.00      0.50      0.67         2
           3       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           6       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         2
           8       1.00      1.00      1.00         2
           9       0.00      0.00      0.00         1
          10       0.75      1.00      0.86         3
          11       0.50      1.00      0.67         1
          12       0.00      0.00      0.00         2
          13       0.67      1.00      0.80         2
          14       1.00      1.00      1.00         3
          16       1.00      0.67      0.80         3
          17       0.50      1.00      0.67         1
          18       0.67      0.67      0.67         3
          19       0.50    

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### SVM - rbf non linear 

In [ ]:
# non-linear model
# using rbf kernel, C=1, default value of gamma

# non-linear model

non_linear = SVC(kernel='rbf')
non_linear.fit(x_train.reshape((x_train.shape[0],42)), np.argmax(labels, axis=1))

# predict
y_pred = non_linear.predict(x_test.reshape((x_test.shape[0],42)))

##### Evaluation

In [ ]:
# confusion matrix and accuracy

# accuracy
print("accuracy:", metrics.accuracy_score(y_true=np.argmax(test_label, axis=1), y_pred=y_pred), "\n")

# cm
print(metrics.confusion_matrix(y_true=np.argmax(test_label, axis=1), y_pred=y_pred))

accuracy: 0.8409090909090909 

[[3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 2 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(np.argmax(test_label, axis=1), y_pred))

              precision    recall  f1-score   support

           0       0.75      1.00      0.86         3
           1       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           6       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         2
           8       1.00      1.00      1.00         2
           9       0.00      0.00      0.00         1
          10       1.00      0.67      0.80         3
          11       0.50      1.00      0.67         1
          12       1.00      0.50      0.67         2
          13       1.00      1.00      1.00         2
          14       1.00      1.00      1.00         3
          16       0.60      1.00      0.75         3
          17       0.33      1.00      0.50         1
          18       0.67      0.67      0.67         3
          19       0.00    

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from joblib import dump, load
s = dump(non_linear,'/content/drive/MyDrive/SVM/model.joblib')
